In [ ]:
root_dir = '/data/ChaochaoData/DSA-DL/HyperMorph/'
workspace = root_dir + 'HM_PS_likert/'
util_dir = '/data/ChaochaoData/DSA-DL/Utilities/'
train_dir = '/data/ChaochaoData/PixShift/DataSets/PaperData/CombinedTrain'
test_dir = '/data/ChaochaoData/PixShift/DataSets/PaperData/CombinedTest'
cm_train_dir = '/data/ChaochaoData/ClearMatch/nifti_predictions/train_dataset'
cm_test_dir = '/data/ChaochaoData/ClearMatch/nifti_predictions/test_dataset'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
# plt.rcParams['animation.ffmpeg_path'] = '/usr/local/lib/python3.8/dist-packages'
import matplotlib.cm as cm
from matplotlib.colors import Normalize

import neurite as ne
import voxelmorph as vxm
import tensorflow as tf

import os, sys, shutil
import nibabel as nib
import cv2
# from scipy import ndimage
# from IPython import display  # Would conflict with Python's display
import IPython
import logging
import warnings

sys.path.insert(0, util_dir)
from utils import *
# from paper_visualization import *

from pixelshifting import generateAffineDSA

warnings.filterwarnings('ignore')
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.experimental.output_all_intermediates(True)
tf.compat.v1.disable_eager_execution()
# tf.compat.v1.enable_eager_execution()

2024-06-02 00:50:59.296654: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Loading BokehJS ...

In [ ]:
folders_train_old = [os.path.join(train_dir, f) for f in sorted(os.listdir(train_dir))
                     if not os.path.isfile(os.path.join(train_dir, f))]
folders_test_old = [os.path.join(test_dir, f) for f in sorted(os.listdir(test_dir))
                    if not os.path.isfile(os.path.join(test_dir, f))]
print(len(folders_train_old), len(folders_test_old))

folders_train_cm = [os.path.join(cm_train_dir, f) for f in sorted(os.listdir(cm_train_dir))
                    if not os.path.isfile(os.path.join(cm_train_dir, f))]
folders_test_cm = [os.path.join(cm_test_dir, f) for f in sorted(os.listdir(cm_test_dir))
                   if not os.path.isfile(os.path.join(cm_test_dir, f))]
print(len(folders_train_cm), len(folders_test_cm))

folders_train = folders_train_old + folders_train_cm
folders_test = folders_test_old + folders_test_cm

print(len(folders_train), len(folders_test))

folders_sel = folders_test

4946 100
100 94
5046 194


In [ ]:
len(folders_test_old), len(folders_test_cm)

(100, 94)

In [ ]:
# model_names = ['Naive', 'MinMax', 'MeanMask', 'FilMask', 'ProbMatte']
image_shape = (512, 512)

def init_hpnet(hp_input):
    x = tf.keras.layers.Dense(32, activation='relu')(hp_input)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    hypernetwork = tf.keras.Model(hp_input, x, name='hypernetwork')
    return hypernetwork

def load_model(model_name):
    if model_name is 'Naive':
        hp_input = tf.keras.Input(shape=[1])
        hypernetwork = init_hpnet(hp_input)
        model = vxm.networks.VxmDense(image_shape, int_steps=0, hyp_model=hypernetwork)
        model_path = os.path.join(root_dir, 'Naive/dsa-hyper-naive.h5')
        model.load_weights(model_path)
    elif model_name is 'MinMax':
        hp_input = tf.keras.Input(shape=[1])
        hypernetwork = init_hpnet(hp_input)
        model = vxm.networks.VxmDense(image_shape, int_steps=0, bidir=True, hyp_model=hypernetwork)
        model_path = os.path.join(root_dir, 'MinMax/dsa_hyper_cycle.h5')
        model.load_weights(model_path)
    elif model_name is 'MeanMask':
        hp_input = tf.keras.Input(shape=[2])
        hypernetwork = init_hpnet(hp_input)
        model = vxm.networks.VxmDense(image_shape, int_steps=0, hyp_model=hypernetwork)
        model_path = os.path.join(root_dir, 'LayerSep-mean-thresh/dsa-hyper-ls.h5')
        model.load_weights(model_path)
    elif model_name is 'FilMask':
        hp_input = tf.keras.Input(shape=[1])
        hypernetwork = init_hpnet(hp_input)
        model = vxm.networks.VxmDense(image_shape, int_steps=0, hyp_model=hypernetwork)
        model_path = os.path.join(root_dir, 'LayerSep-filter/dsa-hyper-ls.h5')
        model.load_weights(model_path)
    elif model_name is 'ProbMatte':
        hp_input = tf.keras.Input(shape=[1])
        hypernetwork = init_hpnet(hp_input)
        model = vxm.networks.VxmDense(image_shape, int_steps=0, hyp_model=hypernetwork)
        model_path = os.path.join(root_dir, 'LayerSep-prob/dsa-hyper-ls.h5')
        model.load_weights(model_path)
    # print(os.path.exists(model_path))
    return model

In [ ]:
def dsa_predict(model_name, xseq_bg, xseq_ct):
    if model_name is 'Naive':
        hp = [0.7]
    elif model_name is 'MinMax':
        hp = [0.5]
    elif model_name is 'MeanMask':
        hp = [0.7, 0]
    elif model_name is 'FilMask':
        hp = [0.7]
    elif model_name is 'ProbMatte':
        hp = [0.7]

    nx = len(xseq_bg)
    hp_seq = np.repeat(np.array([hp]), nx, axis=0)
    inputs = [xseq_bg, xseq_ct, hp_seq]

    model = models[model_name]
    preds = model.predict(inputs, verbose=0)
    xseq_mv = preds[0]

    dsa_seq = xseq_ct - xseq_mv
    return dsa_seq

In [ ]:
def scaling(images):
    images = (images - images.min()) / (images.max() - images.min())
    images = images * 4095.
    return images

def matching(image, matchImage):
    ## Take the center of the frame to avoid issues with collumnation
    offs = 100
    lb = 0+offs
    ub = 511-offs

    firstFrameMean = np.mean(image[0,lb:ub,lb:ub])
    firstFrameMeanMatch = np.mean(matchImage[0,lb:ub,lb:ub])

    image_centered = image - firstFrameMean
    matchImage_centered = matchImage-firstFrameMeanMatch

    ratio_std = np.std(matchImage_centered[:,lb:ub,lb:ub]) / np.std(image_centered[:,lb:ub,lb:ub])
    image_new = image_centered * ratio_std
    image_new = image_new + firstFrameMeanMatch

    maximum = np.amax(matchImage)
    minimum = np.amin(matchImage)
    image_new[image_new > maximum] = maximum
    image_new[image_new < minimum] = minimum
    return image_new

In [ ]:
def load_xray_seq(folders, i_seq=None, glob_norm=False, filename='input.nii'):
    if i_seq is None:
        i_seq = np.random.choice(len(folders))

    path = os.path.join(folders[i_seq], filename)
    xrays_raw = nib.load(path).get_fdata()

    xrays = np.moveaxis(xrays_raw, -1, 0)
    # if glob_norm:
    #     xrays = xrays / 4095.
    # else:
    #     xrays = (xrays - xrays.min()) / (xrays.max() - xrays.min())
    # # xrays = [cv2.resize(xa, dsize=(32, 32), interpolation=cv2.INTER_CUBIC) for xa in xrays]
    # # xrays = [cv2.cvtColor(xa, cv2.COLOR_GRAY2RGB) for xa in xrays]
    # xrays = np.array(xrays, dtype='float32')
    # xrays = xrays[..., None]

    return xrays

In [ ]:
def margin_cropping(xrays, margin=10):
    return xrays[:, margin:-margin, margin:-margin]

In [ ]:
def out_dsa_cases(xrays):
    idx_f0 = 0
    nx = xrays.shape[0]

    print(xrays.shape, xrays.dtype, xrays.min(), xrays.max())
    print(xrays.transpose(1,2,0).shape)
    dsa_affine = generateAffineDSA(xrays)
    dsa_affine = scaling(dsa_affine)
    dsa_affine = margin_cropping(dsa_affine)
    dsa_cases=[dsa_affine]

    xrays = (xrays - xrays.min()) / (xrays.max() - xrays.min())
    xrays = np.array(xrays, dtype='float32')
    xrays = xrays[..., None]

    xseq_bg = np.repeat(xrays[idx_f0][None], nx, axis=0)
    xseq_ct = xrays

    # dsa_org = xseq_ct - xseq_bg  # original dsa
    # dsa_org = scaling(dsa_org.squeeze())
    # dsa_cases = [dsa_org]

    for model_name in models.keys():
        dsa = dsa_predict(model_name, xseq_bg, xseq_ct)  # predicted dsa
        dsa = scaling(dsa.squeeze())
        dsa = margin_cropping(dsa)
        dsa_cases.append(dsa)

    ## Adjust dsa
    # dsa_cases[0] = matching(image=dsa_cases[0], matchImage=dsa_cases[3])
    matchDSA = np.mean(dsa_cases, axis=0)
    for i in range(len(dsa_cases)):
        dsa_cases[i] = matching(image=dsa_cases[i], matchImage=matchDSA)

    return dsa_cases

In [ ]:
affine = [[0, -1, 0, 0],
          [-1, 0, 0, 0],
          [0, 0, 1, 0],
          [0, 0, 0, 1]]

def output_nifti(images, path):
    images = np.moveaxis(images, 0, -1)  # change batch axis
    image = nib.Nifti1Image(images, affine)
    image.to_filename(path)

In [ ]:
def output_single_pool(folders_sel, i_seq=None, prompt=False):
    n_seq = len(folders_sel)
    if i_seq is None:
        i_seq = np.random.randint(n_seq)
    if prompt: print(f'# of seqence: {i_seq}')

    dst_dir = os.path.join( samples_dir, str(i_seq).zfill(3) )
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)

    xrays = load_xray_seq(folders_sel, i_seq)
    nx = len(xrays)

    nx_max = 25
    if nx > nx_max:
        indices = np.linspace(0, nx-1, nx_max, endpoint=True, dtype='int64')
        xrays = xrays[indices]
        nx = nx_max

    dsa_cases = out_dsa_cases(xrays)

    pool_order = np.random.permutation(2)
    if prompt: print(f'pool order: {pool_order}')
    sep_wid = 4
    #row1 = np.concatenate((dsa_cases[pool_order[0]], np.zeros((nx,512,sep_wid)), dsa_cases[pool_order[1]]), axis=2)
    #row2 = np.concatenate((dsa_cases[pool_order[2]], np.zeros((nx,512,sep_wid)), dsa_cases[pool_order[3]]), axis=2)
    #pool_dsa_seq = np.concatenate((row1, np.zeros((nx,sep_wid,row1.shape[-1])), row2), axis=1)
    H, W = dsa_cases[0].shape[1:3]
    pool_dsa_seq = np.concatenate((dsa_cases[pool_order[0]],
                                   np.zeros((nx,H,sep_wid)),
                                   dsa_cases[pool_order[1]]), axis=2)
    if prompt: print(f'pool image shape: {pool_dsa_seq.shape}')

    output_nifti(pool_dsa_seq, os.path.join(dst_dir, 'pool_dsa.nii'))

    with open(os.path.join(dst_dir, "pool_order.txt"), "w") as file:
        for idx in pool_order:
            file.write(" ".join(str(idx)) + "\n") # works with any number of elements in a line

    with open(os.path.join(dst_dir, "src_dir.txt"), "w") as file:
        file.write(folders_sel[i_seq])

## Output test samples

In [ ]:
models = {
    #'Naive': load_model('Naive'),
    #'MinMax': load_model('MinMax'),
    'FilMask': load_model('FilMask')
}

In [ ]:
samples_dir = os.path.join(workspace, 'likert_samples')

if os.path.exists(samples_dir):
    shutil.rmtree(samples_dir)

if not os.path.exists(samples_dir):
    os.makedirs(samples_dir)
    print("folder has been created")

folder has been created


In [ ]:
N_seq = len(folders_sel)
all_idxes = [*range(N_seq)]

old_dir = os.path.join(workspace, 'test_samples')

if os.path.exists(old_dir):
    old_idxes = [int(f) for f in sorted(os.listdir(old_dir))
                if not os.path.isfile(os.path.join(old_dir, f))]
    fil_idxes = [i for i in all_idxes if i not in old_idxes]
else:
    fil_idxes = all_idxes

In [ ]:
tmp_idxes = np.random.choice(len(fil_idxes), size=50, replace=False)
indices_seq = [fil_idxes[i] for i in tmp_idxes]

for (i, i_seq) in enumerate(indices_seq):
    IPython.display.clear_output(wait=True)
    output_single_pool(folders_sel, i_seq=i_seq, prompt=False)
    if (i+1) % 10 != 0:
        print('.', end='')
    else:
        print('.')

(23, 512, 512) float64 399.0 4096.0
(512, 512, 23)
I am on frame 0.
There are 4763 matches.
[[ 1. -0.  0.]
 [ 0.  1.  0.]]
I am on frame 1.
There are 4763 matches.
[[ 1.00004511e+00 -9.37101141e-04  5.93274859e-01]
 [ 9.37101141e-04  1.00004511e+00 -1.12714912e+00]]
I am on frame 2.
There are 4763 matches.
[[ 9.99601971e-01 -1.01736855e-03  9.82672412e-01]
 [ 1.01736855e-03  9.99601971e-01 -1.56770389e+00]]
I am on frame 3.
There are 4763 matches.
[[ 9.99609374e-01 -6.27644413e-04  9.59882453e-01]
 [ 6.27644413e-04  9.99609374e-01 -1.46584005e+00]]
I am on frame 4.
There are 4763 matches.
[[ 9.99212953e-01 -1.28502688e-03  1.18612839e+00]
 [ 1.28502688e-03  9.99212953e-01 -1.67494405e+00]]
I am on frame 5.
There are 4763 matches.
[[ 9.99811968e-01 -1.86190245e-03  1.35334834e+00]
 [ 1.86190245e-03  9.99811968e-01 -2.19568253e+00]]
I am on frame 6.
There are 4763 matches.
[[ 1.00166169e+00 -2.43113257e-04  7.50487137e-01]
 [ 2.43113257e-04  1.00166169e+00 -2.28830975e+00]]
I am on frame

In [ ]:
# output_single_pool(folders_sel, i_seq=134, prompt=True)